# Chapter 7 An exotics engine and the template pattern.

Now that we have a set of components we will built a Monte Carlo pricer for a path dependent exotic option.

For that we will need the following components:
* The generation of the stock price path.
* The generation of cash-flows given a stock price path.
* The discounting and summing of cash-flows for a given path.
* The averaging of the prices over all paths.

For the last point we can use our statistics gatherer, for the second we can create just a pay-off class. The path generation will be a virtual method of the base class (of the exotic engine (an example of the template design pattern)). The main idea of it is that we will have a base class that controls all the steps of the simulation calling virtual methods and then an inhereted class will define those methods. The discounting and summing of cash-flows will be included also in the main engine, since it does not have to know anything about the interest rates (that is handled by pur parameters class).

### The base classes

In [1]:
from copy import deepcopy

In [137]:
class CashFlow:
    def __init__(self, TimeIndex_n ,Amount_ = 0):
        self.Amount = Amount_
        self.TimeIndex = TimeIndex_n

class PathDependent:
    def __init__(self, LookAtTimes_):
        self.__LookAtTimes = LookAtTimes_
        
    def GetLookAtTimes(self):
        return self.__LookAtTimes
    
    def MaxNumberOfCashFlows(self):
        pass
    
    def PossibleCashFlowTimes(self):
        pass
    
    def CashFlows(self, SpotValues, GeneratedFlows):
        pass
        
    def clone(self):
        from copy import deepcopy
        return deepcopy(self)
    
    def __del__(self):
        del self


class ExoticEngine:
    def __init__(self, TheProduct_, r_):
        self.TheProduct = TheProduct_
        self.__r = r_
        self.__Discounts = self.TheProduct.PossibleCashFlowTimes()
        
        
        for i in range(len(self.__Discounts)):
            self.__Discounts[i] = exp(-self.__r.Integral(0,self.__Discounts[i]))
            
        self.__TheseCashFlows = []
        for i in range(self.TheProduct.MaxNumberOfCashFlows()):
            self.__TheseCashFlows.append(CashFlow(0,0))
            
    
    def GetOnePath(self, SpotValues):
        pass
    
    def DoSimulation(self,TheGatherer, NumberOfPaths):
        SpotValues = self.TheProduct.GetLookAtTimes()
        
        self.__TheseCashFlows = []
        for i in range(self.TheProduct.MaxNumberOfCashFlows()):
            self.__TheseCashFlows.append(CashFlow(0,0))
            
        
        for i in range(NumberOfPaths):
            self.GetOnePath(SpotValues)
            thisValue = self.DoOnePath(SpotValues)
            TheGatherer.DumpOneResult(thisValue)
    
    def __del__(self):
        del self
        
    def DoOnePath(self, SpotValues):
        NumberFlows = self.TheProduct.CashFlows(SpotValues,self.__TheseCashFlows)
        Value = 0
        for i in range(NumberFlows):
            Value += self.__TheseCashFlows[i].Amount*self.__Discounts[self.__TheseCashFlows[i].TimeIndex]
            
        return Value 

In [138]:
class ExoticBSEngine(ExoticEngine):
    def __init__(self,TheProduct_, R_, D_, Vol_, TheGenerator_, Spot_):
        super().__init__(TheProduct_, R_)
        self.TheProduct = TheProduct_ 
        self.__r = R_ 
        self.__TheGenerator = TheGenerator_        
        Times = self.TheProduct.GetLookAtTimes()
        self.__NumberOfTimes = len(Times)
        
        self.__Discounts = self.TheProduct.PossibleCashFlowTimes()
        
        
        for i in range(len(self.__Discounts)):
            self.__Discounts[i] = exp(-self.__r.Integral(0,self.__Discounts[i]))
            
        self.__TheseCashFlows = []
        for i in range(self.TheProduct.MaxNumberOfCashFlows()):
            self.__TheseCashFlows.append(CashFlow(0,0))
            
               
        self.__TheGenerator.ResetDimensionality(self.__NumberOfTimes)        
        self.__Drifts  = np.zeros(self.__NumberOfTimes,float)
        self.__StandardDeviations = np.zeros(self.__NumberOfTimes,float)
        
        Variance = Vol_.IntegralSquare(0,Times[0])
        
        self.__Drifts[0] = R_.Integral(0,Times[0]) - D_.Integral(0,Times[0])-0.5*Variance
        
        self.__StandardDeviations[0] = sqrt(Variance)
        
        for j in range(1,self.__NumberOfTimes,1):
            thisVariance = Vol_.IntegralSquare(Times[j-1],Times[j])
            self.__Drifts[j] = R_.Integral(Times[j-1],times[j])-D_.Integral(Times[j-1],
                                                                            Times[j]) -0.5*thisVariance
                
            self.__StandardDeviations[j] = sqrt(thisVariance)
            
        self.__LogSpot = log(Spot_)
        self.__Variates = np.zeros(self.__NumberOfTimes,float)
        

        
    def GetOnePath(self, SpotValues):
        self.__Variates = self.__TheGenerator.GetGaussians(self.__Variates)
        CurrentLogSpot = self.__LogSpot
        for j in range(self.__NumberOfTimes):
            CurrentLogSpot += self.__Drifts[j]
            CurrentLogSpot += self.__StandardDeviations[j]*self.__Variates[j]
            SpotValues[j] = exp(CurrentLogSpot)        

### An arithmetic Asian option

In [4]:
class PathDependentAsian(PathDependent):
    def __init__(self, LookAtTimes_, DeliveryTime_, ThePayOff_, ):
        super().__init__(LookAtTimes_)
        self.__DeliveryTime = DeliveryTime_
        self.__ThePayOff = ThePayOff_
        self.__NumberOfTimes = len(LookAtTimes_)
        
    def MaxNumberOfCashFlows(self):
        return 1
    
    def PossibleCashFlowTimes(self):
        import numpy as np
        tmp = np.zeros(1,float)
        tmp[0] = self.__DeliveryTime
        return tmp
    
    def CashFlows(self, SpotValues, GeneratedFlows):
        sum_ = np.sum(SpotValues)
        mean = sum_ / self.__NumberOfTimes
        GeneratedFlows[0].timeIndex = 0
        GeneratedFlows[0].Amount = self.__ThePayOff(mean)
        return 1

### Putting it all together

In [5]:
from random import random
from math import exp, sqrt, log
import numpy as np

In [6]:
class PayOff():
    def __init__(self):
        pass
    
    def __call__(self,Spot):
        pass    

class PayOffBridge():        
    def __del__(self):
        del self
        
    def copy(self,InnerPayOff):
        from copy import deepcopy
        self = deepcopy(InnerPayOff)
        
class PayOffCall(PayOff,PayOffBridge):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(Spot - self.__Strike,0)    

In [7]:
class StatisticsMC:
    def __init__(self):
        pass
    
    def GetResultsSoFar(self):
        pass
    
    def DumpOneResult(self,result):
        pass        
  
    def clone(self):
        pass
    
    def __del__(self):
        pass   
    

class StatisticsMean(StatisticsMC):
    def __init__(self):
        self.__RunningSum = 0
        self.__PathsDone = 0
        
    def GetResultsSoFar(self): 
        Results = [[0]]
        Results[0][0] = self.__RunningSum / self.__PathsDone
        return Results
    
    def DumpOneResult(self,result):
        self.__PathsDone += 1
        self.__RunningSum += result        
        
    def clone(self):
        return deepcopy(self)
    
    def __del__(self):
        del self
        
        
class ConvergenceTable(StatisticsMC):
    def __init__(self, Inner_):
        self.__Inner = Inner_
        self.__ResultsSoFar = []
        self.__StoppingPoint = 2
        self.__PathsDone = 0
    
    def clone(self):
        return deepcopy(self)
    
    def DumpOneResult(self,result):
        self.__Inner.DumpOneResult(result)
        self.__PathsDone += 1
        
        
        if self.__PathsDone == self.__StoppingPoint:
            self.__StoppingPoint *=2
            thisResult = self.__Inner.GetResultsSoFar()            
            
            for i in range(len(thisResult)):
                thisResult[i].append(self.__PathsDone)
                self.__ResultsSoFar.append(thisResult[i]) 

            
    
    def GetResultsSoFar(self):
        
        tmp = self.__ResultsSoFar 
        
        if self.__PathsDone*2 != self.__StoppingPoint:
            thisResult = self.__Inner.GetResultsSoFar()
            
            for i in range(len(thisResult)):
                thisResult[i].append(self.__PathsDone)
                tmp.append(thisResult[i]) 
                
        return tmp
                

In [8]:
class ParametersInner:
    def __init__(self):
        pass
    
    def Integral(self,time1, time2):
        pass
    
    def IntegralSquare(self,time1,time2):
        pass

class Parameters:    
    def RootMeanSquare(self,time1,time2):
        total = self.Integral(time1,time2)
        return total/(time2-time1)
    
    def Mean(self,time1, time2):
        total = self.Integral(time1,time2)
        return total/(time2-time1)
    
    def copy(self,original):
        from copy import deepcopy
        self = deepcopy(InnerPayOff) 
        
    def __del__(self):
        del self    
        
class ParametersConstant(ParametersInner,Parameters):
    def __init__(self,constant):
        self.__Constant = constant
        self.__ConstantSquare = self.__Constant*self.__Constant
    
    def Integral(self, time1, time2):
        return (time2 - time1)*self.__Constant
    
    def IntegralSquare(self, time1, time2):
        return (time2 - time1)*self.__ConstantSquare   


In [9]:
class Generator:
    def __init__(self,Dimensionality_ = 1):
        self.__Dimensionality = Dimensionality_
        
    def ResetDimensionality(self,Dimensionality_):
        self.__Dimensionality = Dimensionality_
    
    def GetGaussians(self,Variates):
        Variates = np.random.normal(size = self.__Dimensionality)
        return Variates

We will first test it with an arithmetic asian option that is no path dependent. The correct price is 5.13.

In [10]:
Expiry = 1
Strike = 50
Spot = 50
Vol = 0.2309
r = 0.1
d = 0.0633
NumberOfPaths = 1000000
NumberOfDates = 1

In [11]:
thePayOff = PayOffCall(Strike)
times = np.zeros(NumberOfDates)

for i in range(len(times)):
    times[i] = (i+1)*Expiry/NumberOfDates
    
VolParam  = ParametersConstant(Vol)
rParam = ParametersConstant(r)
dParam = ParametersConstant(d)

theOption = PathDependentAsian(times, Expiry, thePayOff)

gatherer = StatisticsMean()
gathererTwo = ConvergenceTable(gatherer)

generator = Generator(NumberOfDates)

theEngine = ExoticBSEngine(theOption, rParam, dParam, VolParam, generator, Spot)

theEngine.DoSimulation(gathererTwo, NumberOfPaths)

results = gathererTwo.GetResultsSoFar()

print('For the Asian call price the results are')

for i in range(len(results)):
    for j in range(len(results[i])):
        print(results[i][j])

For the Asian call price the results are
13.732430956446157
2
17.898395416921183
4
9.21273981329508
8
5.312933832169964
16
5.038650617577289
32
4.980412777335529
64
5.60341456535641
128
6.215197417209491
256
5.800192398090293
512
5.560482682875568
1024
5.344911271549051
2048
5.297811025224769
4096
5.236674823505501
8192
5.184445011659788
16384
5.174740254866614
32768
5.145687255841427
65536
5.133713998312835
131072
5.133609152473891
262144
5.147376079741058
524288
5.14230417960901
1000000


### Exercises

**Exercise 7.1** Write a class to do geometric Asian options.

We just have to modify the CashFlows method changing the arithmetic mean for the geometric one.

In [71]:
class PathDependentAsianGeometric(PathDependent):
    def __init__(self, LookAtTimes_, DeliveryTime_, ThePayOff_, ):
        super().__init__(LookAtTimes_)
        self.__DeliveryTime = DeliveryTime_
        self.__ThePayOff = ThePayOff_
        self.__NumberOfTimes = len(LookAtTimes_)
        
    def MaxNumberOfCashFlows(self):
        return 1
    
    def PossibleCashFlowTimes(self):
        import numpy as np
        tmp = np.zeros(1,float)
        tmp[0] = self.__DeliveryTime
        return tmp
    
    def CashFlows(self, SpotValues, GeneratedFlows):
        prod_ = np.prod(SpotValues)
        mean = pow(prod_,1/self.__NumberOfTimes)
        GeneratedFlows[0].timeIndex = 0
        GeneratedFlows[0].Amount = self.__ThePayOff(mean)
        return 1

If we want to be more reusable we could do the following.

In [72]:
class PathDependentAsian(PathDependent):
    def __init__(self, LookAtTimes_, DeliveryTime_, ThePayOff_):
        super().__init__(LookAtTimes_)
        self._DeliveryTime = DeliveryTime_
        self._ThePayOff = ThePayOff_
        self._NumberOfTimes = len(LookAtTimes_)
        
    def MaxNumberOfCashFlows(self):
        return 1
    
    def PossibleCashFlowTimes(self):
        import numpy as np
        tmp = np.zeros(1,float)
        tmp[0] = self._DeliveryTime
        return tmp
        
    def CashFlows(self, SpotValues, GeneratedFlows):
        pass

In [73]:
class PathDependentAsianGeometric(PathDependentAsian):
    def __init__(self, LookAtTimes_, DeliveryTime_, ThePayOff_ ):
        super().__init__(LookAtTimes_, DeliveryTime_, ThePayOff_)
    
    def CashFlows(self, SpotValues, GeneratedFlows):
        prod_ = np.prod(SpotValues)
        mean = pow(prod_,1/self._NumberOfTimes)
        GeneratedFlows[0].timeIndex = 0
        GeneratedFlows[0].Amount = self._ThePayOff(mean)
        return 1

In [15]:
class PathDependentAsianArithmetic(PathDependentAsian):
    def __init__(self, LookAtTimes_, DeliveryTime_, ThePayOff_ ):
        super().__init__(LookAtTimes_, DeliveryTime_, ThePayOff_)
    
    def CashFlows(self, SpotValues, GeneratedFlows):
        sum_ = np.sum(SpotValues)
        mean = sum_/self._NumberOfTimes
        GeneratedFlows[0].timeIndex = 0
        GeneratedFlows[0].Amount = self._ThePayOff(mean)
        return 1

In [74]:
Expiry = 1
Strike = 50
Spot = 50
Vol = 0.2309
r = 0.1
d = 0.0633
NumberOfPaths = 1000000
NumberOfDates = 12

In [75]:
%%time
thePayOff = PayOffCall(Strike)
times = np.zeros(NumberOfDates)

for i in range(len(times)):
    times[i] = (i+1)*Expiry/NumberOfDates
    
VolParam  = ParametersConstant(Vol)
rParam = ParametersConstant(r)
dParam = ParametersConstant(d)

theOption = PathDependentAsianGeometric(times, Expiry, thePayOff)

gatherer = StatisticsMean()
gathererTwo = ConvergenceTable(gatherer)

generator = Generator(NumberOfDates)

theEngine = ExoticBSEngine(theOption, rParam, dParam, VolParam, generator, Spot)

theEngine.DoSimulation(gathererTwo, NumberOfPaths)

results = gathererTwo.GetResultsSoFar()

print('For the Asian call price the results are')

for i in range(len(results)):
    for j in range(len(results[i])):
        print(results[i][j])

For the Asian call price the results are
5.5980767650593
2
2.79903838252965
4
3.4760037046201107
8
4.679515558677616
16
3.3038393217575504
32
2.763343080312859
64
2.7207256830871733
128
3.074625984026488
256
3.0932756463835167
512
3.104169519192656
1024
3.136539316565668
2048
2.967192976269604
4096
2.9812322265246305
8192
2.940999575946381
16384
2.974910035983124
32768
2.9709697845495895
65536
2.956104778261415
131072
2.951998719252699
262144
2.9390196862934928
524288
2.9414783417731423
1000000
Wall time: 35.2 s


**Exercise 7.2** Write a class to do discrete knock-out options that pay a rebate at the time of rebate.

**Exercise 7.3** Rewrite the classes here so that they pass the logs of spot values around instead of the spot values. Show that the discrete barrier option and the geometric Asian need fewer exponentiations.

In [76]:
class ExoticEngine:
    def __init__(self, TheProduct_, r_):
        self.TheProduct = TheProduct_
        self.__r = r_
        self.__Discounts = self.TheProduct.PossibleCashFlowTimes()
        
        
        for i in range(len(self.__Discounts)):
            self.__Discounts[i] = exp(-self.__r.Integral(0,self.__Discounts[i]))
            
        self.__TheseCashFlows = []
        for i in range(self.TheProduct.MaxNumberOfCashFlows()):
            self.__TheseCashFlows.append(CashFlow(0,0))
            
    
    def GetOnePath(self, logSpotValues):
        pass
    
    def DoSimulation(self,TheGatherer, NumberOfPaths):
        logSpotValues = np.array(self.TheProduct.GetLookAtTimes())
        
        self.__TheseCashFlows = []
        for i in range(self.TheProduct.MaxNumberOfCashFlows()):
            self.__TheseCashFlows.append(CashFlow(0,0))
            
        
        for i in range(NumberOfPaths):
            self.GetOnePath(logSpotValues)
            thisValue = self.DoOnePath(logSpotValues)
            TheGatherer.DumpOneResult(thisValue)
    
    def __del__(self):
        del self
        
    def DoOnePath(self, logSpotValues):
        NumberFlows = self.TheProduct.CashFlows(logSpotValues,self.__TheseCashFlows)
        Value = 0
        for i in range(NumberFlows):
            Value += self.__TheseCashFlows[i].Amount*self.__Discounts[self.__TheseCashFlows[i].TimeIndex]
            
        return Value 

In [77]:
class ExoticBSEngine(ExoticEngine):
    def __init__(self,TheProduct_, R_, D_, Vol_, TheGenerator_, Spot_):
        super().__init__(TheProduct_, R_)
        self.TheProduct = TheProduct_ 
        self.__r = R_ 
        self.__TheGenerator = TheGenerator_        
        Times = self.TheProduct.GetLookAtTimes()
        self.__NumberOfTimes = len(Times)
        
        self.__Discounts = self.TheProduct.PossibleCashFlowTimes()
        
        
        for i in range(len(self.__Discounts)):
            self.__Discounts[i] = exp(-self.__r.Integral(0,self.__Discounts[i]))
            
        self.__TheseCashFlows = []
        for i in range(self.TheProduct.MaxNumberOfCashFlows()):
            self.__TheseCashFlows.append(CashFlow(0,0))
            
               
        self.__TheGenerator.ResetDimensionality(self.__NumberOfTimes)        
        self.__Drifts  = np.zeros(self.__NumberOfTimes,float)
        self.__StandardDeviations = np.zeros(self.__NumberOfTimes,float)
        
        Variance = Vol_.IntegralSquare(0,Times[0])
        
        self.__Drifts[0] = R_.Integral(0,Times[0]) - D_.Integral(0,Times[0])-0.5*Variance
        
        self.__StandardDeviations[0] = sqrt(Variance)
        
        for j in range(1,self.__NumberOfTimes,1):
            thisVariance = Vol_.IntegralSquare(Times[j-1],Times[j])
            self.__Drifts[j] = R_.Integral(Times[j-1],times[j])-D_.Integral(Times[j-1],
                                                                            Times[j]) -0.5*thisVariance
                
            self.__StandardDeviations[j] = sqrt(thisVariance)
            
        self.__LogSpot = log(Spot_)
        self.__Variates = np.zeros(self.__NumberOfTimes,float)
        

        
    def GetOnePath(self, logSpotValues):
        self.__Variates = self.__TheGenerator.GetGaussians(self.__Variates)
        CurrentLogSpot = self.__LogSpot
        for j in range(self.__NumberOfTimes):
            CurrentLogSpot += self.__Drifts[j]
            CurrentLogSpot += self.__StandardDeviations[j]*self.__Variates[j]
            logSpotValues[j] = CurrentLogSpot     

In [60]:
class PathDependentAsian(PathDependent):
    def __init__(self, LookAtTimes_, DeliveryTime_, ThePayOff_):
        super().__init__(LookAtTimes_)
        self._DeliveryTime = DeliveryTime_
        self._ThePayOff = ThePayOff_
        self._NumberOfTimes = len(LookAtTimes_)
        
    def MaxNumberOfCashFlows(self):
        return 1
    
    def PossibleCashFlowTimes(self):
        import numpy as np
        tmp = np.zeros(1,float)
        tmp[0] = self._DeliveryTime
        return tmp
        
    def CashFlows(self, logSpotValues, GeneratedFlows):
        pass

In [78]:
class PathDependentAsianGeometric(PathDependentAsian):
    def __init__(self, LookAtTimes_, DeliveryTime_, ThePayOff_ ):
        super().__init__(LookAtTimes_, DeliveryTime_, ThePayOff_)
    
    def CashFlows(self, logSpotValues, GeneratedFlows):
        sum_ = np.sum(logSpotValues)
        mean = exp(sum_ / self._NumberOfTimes)
        GeneratedFlows[0].timeIndex = 0
        GeneratedFlows[0].Amount = self._ThePayOff(mean)
        return 1

In [79]:
Expiry = 1
Strike = 50
Spot = 50
Vol = 0.2309
r = 0.1
d = 0.0633
NumberOfPaths = 1000000
NumberOfDates = 12

In [80]:
%%time
thePayOff = PayOffCall(Strike)
times = np.zeros(NumberOfDates)

for i in range(len(times)):
    times[i] = (i+1)*Expiry/NumberOfDates
    
VolParam  = ParametersConstant(Vol)
rParam = ParametersConstant(r)
dParam = ParametersConstant(d)

theOption = PathDependentAsianGeometric(times, Expiry, thePayOff)

gatherer = StatisticsMean()
gathererTwo = ConvergenceTable(gatherer)

generator = Generator(NumberOfDates)

theEngine = ExoticBSEngine(theOption, rParam, dParam, VolParam, generator, Spot)

theEngine.DoSimulation(gathererTwo, NumberOfPaths)

results = gathererTwo.GetResultsSoFar()

print('For the Asian call price the results are')

for i in range(len(results)):
    for j in range(len(results[i])):
        print(results[i][j])

For the Asian call price the results are
0.4952098439865451
2
0.31653830124295873
4
2.494355781921428
8
2.779655520574221
16
3.0041734713959585
32
2.867837288824354
64
3.036337378135285
128
2.6058082458537237
256
2.6724857421939543
512
2.8285542127725667
1024
2.8342607567277973
2048
2.8356497856221288
4096
2.855413295911564
8192
2.896759103449885
16384
2.9084441214907515
32768
2.919646836657488
65536
2.929744089735346
131072
2.919817985467737
262144
2.9393814851910607
524288
2.9449717395442794
1000000
Wall time: 34.1 s


As we can see both results are very similar (they differ because of the number of the randomness, but there is a time difference).

**Exercise 7.4** Implement an  engine for pricing when the spot price is normal instead of log-normal.

We just have to eliminate the all the logs in the engine.

In [ ]:
class ExoticNormalEngine:
    def __init__(self, TheProduct_, r_):
        self.TheProduct = TheProduct_
        self.__r = r_
        self.__Discounts = self.TheProduct.PossibleCashFlowTimes()
        
        
        for i in range(len(self.__Discounts)):
            self.__Discounts[i] = exp(-self.__r.Integral(0,self.__Discounts[i]))
            
        self.__TheseCashFlows = []
        for i in range(self.TheProduct.MaxNumberOfCashFlows()):
            self.__TheseCashFlows.append(CashFlow(0,0))
            
    
    def GetOnePath(self, SpotValues):
        pass
    
    def DoSimulation(self,TheGatherer, NumberOfPaths):
        SpotValues = np.array(self.TheProduct.GetLookAtTimes())
        
        self.__TheseCashFlows = []
        for i in range(self.TheProduct.MaxNumberOfCashFlows()):
            self.__TheseCashFlows.append(CashFlow(0,0))
            
        
        for i in range(NumberOfPaths):
            self.GetOnePath(SpotValues)
            thisValue = self.DoOnePath(SpotValues)
            TheGatherer.DumpOneResult(thisValue)
    
    def __del__(self):
        del self
        
    def DoOnePath(self, SpotValues):
        NumberFlows = self.TheProduct.CashFlows(SpotValues,self.__TheseCashFlows)
        Value = 0
        for i in range(NumberFlows):
            Value += self.__TheseCashFlows[i].Amount*self.__Discounts[self.__TheseCashFlows[i].TimeIndex]
            
        return Value 

In [ ]:
class ExoticNormalBSEngine(ExoticEngine):
    def __init__(self,TheProduct_, R_, D_, Vol_, TheGenerator_, Spot_):
        super().__init__(TheProduct_, R_)
        self.TheProduct = TheProduct_ 
        self.__r = R_ 
        self.__TheGenerator = TheGenerator_        
        Times = self.TheProduct.GetLookAtTimes()
        self.__NumberOfTimes = len(Times)
        
        self.__Discounts = self.TheProduct.PossibleCashFlowTimes()
        
        
        for i in range(len(self.__Discounts)):
            self.__Discounts[i] = exp(-self.__r.Integral(0,self.__Discounts[i]))
            
        self.__TheseCashFlows = []
        for i in range(self.TheProduct.MaxNumberOfCashFlows()):
            self.__TheseCashFlows.append(CashFlow(0,0))
            
               
        self.__TheGenerator.ResetDimensionality(self.__NumberOfTimes)        
        self.__Drifts  = np.zeros(self.__NumberOfTimes,float)
        self.__StandardDeviations = np.zeros(self.__NumberOfTimes,float)
        
        Variance = Vol_.IntegralSquare(0,Times[0])
        
        self.__Drifts[0] = R_.Integral(0,Times[0]) - D_.Integral(0,Times[0])-0.5*Variance
        
        self.__StandardDeviations[0] = sqrt(Variance)
        
        for j in range(1,self.__NumberOfTimes,1):
            thisVariance = Vol_.IntegralSquare(Times[j-1],Times[j])
            self.__Drifts[j] = R_.Integral(Times[j-1],times[j])-D_.Integral(Times[j-1],
                                                                            Times[j]) -0.5*thisVariance
                
            self.__StandardDeviations[j] = sqrt(thisVariance)
            
        self.__Spot = Spot_
        self.__Variates = np.zeros(self.__NumberOfTimes,float)
        

        
    def GetOnePath(self, SpotValues):
        self.__Variates = self.__TheGenerator.GetGaussians(self.__Variates)
        CurrentSpot = self.__Spot
        for j in range(self.__NumberOfTimes):
            CurrentSpot += self.__Drifts[j]
            CurrentSpot += self.__StandardDeviations[j]*self.__Variates[j]
            SpotValues[j] = CurrentSpot     

**Exercise 7.5** Write a class that pays the difference in pay-offs of two arbitrary path dependent derivatives.

Since we have to different and arbitrary path dependent derivatives we will need two different engines because they can be from different underlying instruments.

In [130]:
class TwoDifference:
    def __init__(self, Derivative1_, Derivative2_, NumberOfDates1_, NumberOfDates2_, NumberOfPaths_,
                rParam1_, dParam1_, VolParam1_, Spot1_,
                rParam2_, dParam2_, VolParam2_, Spot2_):
        self.generator1 = Generator(NumberOfDates1_)
        self.generator2 = Generator(NumberOfDates2_)
        self.gatherer1 = StatisticsMean()
        self.gathererConvergence1 = ConvergenceTable(self.gatherer1)
        self.gatherer2 = StatisticsMean()
        self.gathererConvergence2 = ConvergenceTable(self.gatherer2)
        self.__NumberOfPaths = NumberOfPaths_
        self.__Engine1 = ExoticBSEngine(Derivative1_, rParam1_, dParam1_, VolParam1_, self.generator1, Spot1_)
        self.__Engine2 = ExoticBSEngine(Derivative2_, rParam2_, dParam2_, VolParam2_, self.generator2, Spot2_)

    
    def DoSimulation(self):
        self.__Engine1.DoSimulation(self.gathererConvergence1, self.__NumberOfPaths)
        self.__Engine2.DoSimulation(self.gathererConvergence2, self.__NumberOfPaths)
        
    def Results(self):
        self.DoSimulation()
        results1 = self.gathererConvergence1.GetResultsSoFar()
        results2 = self.gathererConvergence2.GetResultsSoFar()
        return [[results1[i][0]-results2[i][0],results1[i][1]] for i in range(len(results1))]

In [139]:
Expiry = 1
Strike = 50
Spot = 50
Vol = 0.2
r = 0.1
d = 0
NumberOfPaths = 3000000
NumberOfDates = 1

In [140]:
thePayOff = PayOffCall(Strike)
times = np.zeros(NumberOfDates)

for i in range(len(times)):
    times[i] = (i+1)*Expiry/NumberOfDates
    
VolParam  = ParametersConstant(Vol)
rParam = ParametersConstant(r)
dParam = ParametersConstant(d)

theOption = PathDependentAsianArithmetic(times, Expiry, thePayOff)

difference = TwoDifference(theOption, theOption, NumberOfDates, NumberOfDates, 
                           NumberOfPaths,rParam, dParam, VolParam, Spot, rParam, dParam, VolParam, Spot)

results = difference.Results()

print('For the Asian call price the results are')

for i in range(len(results)):
    for j in range(len(results[i])):
        print(results[i][j])

For the Asian call price the results are
1.5509254208623826
2
5.504751639937849
4
4.79426166579759
8
0.1952799923994153
16
2.207909013582115
32
2.0982510037619573
64
1.979611553495972
128
1.0436758293260056
256
0.5559523144059639
512
0.25769154094857516
1024
0.027654072553674247
2048
0.14057957065861526
4096
0.06770532438299437
8192
0.013737173627094812
16384
-0.02234382740520946
32768
-0.0005363948730296997
65536
0.014155619770725103
131072
0.004598987540844313
262144
0.0055055146464084714
524288
0.008912977070389516
1048576
-0.001605586411341342
2097152
0.006927030533707246
3000000


Since it is the same derivative product it should have a payoff near 0. In this case it is 0.006.